In [5]:
import requests
import json
import time
from openpyxl import Workbook

class BusinessDiscovery:
    def __init__(self, prefix, ig_user_id, ig_creator_username, access_token, creator_field='followers_count,media_count',
                 media_field='caption,comments_count,like_count', proxies=None, time_sleep=1):
        self.url = '%s/%s' % (prefix, ig_user_id)
        self.access_token = access_token
        self.ig_creator_username = ig_creator_username
        self.creator_field = creator_field
        self.media_field = media_field
        self.proxies=proxies
        self.time_sleep = time_sleep

        self.xlsx_head = media_field.split(',')
        self.xlsx_head.append('id')

    def build_params(self, after=None):
        after_str = '.after(%s)' % after if after else ''
        params = {'fields': 'business_discovery.username(%s){%s,media%s{%s}}' % (self.ig_creator_username,
                                                                               self.creator_field,
                                                                               after_str,
                                                                               self.media_field),
                'access_token': self.access_token}
        return params

    def request_by_params(self, params):
        print(self.url, params)
        ret = requests.get(self.url, params=params, proxies=self.proxies)
        return json.loads(ret.content)

    @staticmethod
    def get_paging_cursor(content, direction='after'):
        paging_cursors = content['business_discovery']['media']['paging']['cursors']
        if direction in paging_cursors:
            return paging_cursors[direction]
        else:
            return None

    def write_media_data(self, content, xlsx_sheet):
        media_data = content['business_discovery']['media']['data']
        for media in media_data:
            xlsx_sheet.append([media.get(key, '') for key in self.xlsx_head])

    def request_all_posts_to_xlsx(self, xlsx_path, need_xlsx_head=True):
        wb = Workbook()
        xlsx_sheet = wb.active
        if need_xlsx_head:
            xlsx_sheet.append(self.xlsx_head)

        params = self.build_params()
        content = self.request_by_params(params)
        self.write_media_data(content, xlsx_sheet)

        followers_count = content['business_discovery']['followers_count']
        media_count = content['business_discovery']['media_count']

        if 'paging' in content['business_discovery']['media']:
            next_paging_cursor = self.get_paging_cursor(content)
            while next_paging_cursor:
                params = self.build_params(after=next_paging_cursor)
                content = self.request_by_params(params)
                self.write_media_data(content, xlsx_sheet)

                next_paging_cursor = self.get_paging_cursor(content)
                time.sleep(self.time_sleep)

        wb.save(xlsx_path)

        print('ig_creator_username:', self.ig_creator_username)
        print('followers_count:', followers_count)
        print('media_count:', media_count)
        print('All posts are saving to %s!' % xlsx_path)

In [6]:
ig_user_id = '17841405419489636'
# explorer里generate的access token
token = 'EAAJGCVkgEywBAOY25RaCyHyTavpbp8Rob3Sj2qpmDreBnIoerz6CECHOq31zOP0JdLAw0ATdAGIEkhImHkZA6nNsHQblNB75GRwq2vS2669ZBwLZAjo46oNvMxMvTzcNKKM8XbUgrecMpZCU13XfJZBYrj4Jmj4jYcoHZAilZAB7HzZCZBXnHrQ3bc49MhWjIiIyWNlCRqXRbP3BZBbpBMG5eNfbbrZAU2SwFajYUZC5uQZC3TT18lAbE2pIx'
temp_proxies = None
# temp_proxies = {'https': '127.0.0.1:7890'}

bd = BusinessDiscovery(prefix='https://graph.facebook.com/v12.0',
                       ig_user_id=ig_user_id,
                       ig_creator_username='aespa_official',
                       access_token=token,
                       proxies=temp_proxies)
bd.request_all_posts_to_xlsx('ins_aespa.xlsx')

https://graph.facebook.com/v12.0/17841405419489636 {'fields': 'business_discovery.username(aespa_official){followers_count,media_count,media{caption,comments_count,like_count}}', 'access_token': 'EAAJGCVkgEywBAOY25RaCyHyTavpbp8Rob3Sj2qpmDreBnIoerz6CECHOq31zOP0JdLAw0ATdAGIEkhImHkZA6nNsHQblNB75GRwq2vS2669ZBwLZAjo46oNvMxMvTzcNKKM8XbUgrecMpZCU13XfJZBYrj4Jmj4jYcoHZAilZAB7HzZCZBXnHrQ3bc49MhWjIiIyWNlCRqXRbP3BZBbpBMG5eNfbbrZAU2SwFajYUZC5uQZC3TT18lAbE2pIx'}
https://graph.facebook.com/v12.0/17841405419489636 {'fields': 'business_discovery.username(aespa_official){followers_count,media_count,media.after(QVFIUm5NSWpiOU9MZAnlTODBOTnlTalN5MHRmb1NBeVJqaG9UQmxHNjZAFVmdPSU9rM20tUTYwajl2YktoTmpSaVdEYktIVzhRRlY0cjZAtYTVvM2xsVGN3YnRn){caption,comments_count,like_count}}', 'access_token': 'EAAJGCVkgEywBAOY25RaCyHyTavpbp8Rob3Sj2qpmDreBnIoerz6CECHOq31zOP0JdLAw0ATdAGIEkhImHkZA6nNsHQblNB75GRwq2vS2669ZBwLZAjo46oNvMxMvTzcNKKM8XbUgrecMpZCU13XfJZBYrj4Jmj4jYcoHZAilZAB7HzZCZBXnHrQ3bc49MhWjIiIyWNlCRqXRbP3BZBbpBMG5eNf

https://graph.facebook.com/v12.0/17841405419489636 {'fields': 'business_discovery.username(aespa_official){followers_count,media_count,media.after(QVFIUkxTVjhKVzl2ajVUUGJ6djc3NHlZAZAXNjNEw0ZAC04bU8xU212S3lYSF9FZAWsyZATVBdF9oTGU3OUR5NVJpYnZA0ZA2x2VV9rOHN1cWNvek5GcWZAraVBtUThn){caption,comments_count,like_count}}', 'access_token': 'EAAJGCVkgEywBAOY25RaCyHyTavpbp8Rob3Sj2qpmDreBnIoerz6CECHOq31zOP0JdLAw0ATdAGIEkhImHkZA6nNsHQblNB75GRwq2vS2669ZBwLZAjo46oNvMxMvTzcNKKM8XbUgrecMpZCU13XfJZBYrj4Jmj4jYcoHZAilZAB7HzZCZBXnHrQ3bc49MhWjIiIyWNlCRqXRbP3BZBbpBMG5eNfbbrZAU2SwFajYUZC5uQZC3TT18lAbE2pIx'}
https://graph.facebook.com/v12.0/17841405419489636 {'fields': 'business_discovery.username(aespa_official){followers_count,media_count,media.after(QVFIUktRSmxKVWNHNXN2Ym5ESjFrTjRFd1JQS0lqMXk1NlIxTGtpalc3RzNhZA2h0OEFjeE5PVlgyd2J3UjFSMlI0bWJ2LUc3bWF1b2g2QWdIbjJIclBRdEtB){caption,comments_count,like_count}}', 'access_token': 'EAAJGCVkgEywBAOY25RaCyHyTavpbp8Rob3Sj2qpmDreBnIoerz6CECHOq31zOP0JdLAw0ATdAGIEkhImHkZA6

https://graph.facebook.com/v12.0/17841405419489636 {'fields': 'business_discovery.username(aespa_official){followers_count,media_count,media.after(QVFIUlpqWDdUS3A5OXI3bExPNUNRSnQxa3psYlZAyaUpBT3oxejBEMmhrYmE5VnZAEeUw5VDFuNmtpQlkzM1JaeHlBUnhaOGJ4c2dhOXhkenM0UmU5UVg0M3Jn){caption,comments_count,like_count}}', 'access_token': 'EAAJGCVkgEywBAOY25RaCyHyTavpbp8Rob3Sj2qpmDreBnIoerz6CECHOq31zOP0JdLAw0ATdAGIEkhImHkZA6nNsHQblNB75GRwq2vS2669ZBwLZAjo46oNvMxMvTzcNKKM8XbUgrecMpZCU13XfJZBYrj4Jmj4jYcoHZAilZAB7HzZCZBXnHrQ3bc49MhWjIiIyWNlCRqXRbP3BZBbpBMG5eNfbbrZAU2SwFajYUZC5uQZC3TT18lAbE2pIx'}
https://graph.facebook.com/v12.0/17841405419489636 {'fields': 'business_discovery.username(aespa_official){followers_count,media_count,media.after(QVFIUm5QeUhKWVVHMDFYc3pSN05xY2c1dGxETTBoVG4zcHNMSjFDYy1oMnZAaa08wVENlRUJIOFBXcS1UUV9JRGNpOE1OODNSYUs0MExlcEEwdkRlb2hvWjNB){caption,comments_count,like_count}}', 'access_token': 'EAAJGCVkgEywBAOY25RaCyHyTavpbp8Rob3Sj2qpmDreBnIoerz6CECHOq31zOP0JdLAw0ATdAGIEkhImHkZA6nNsHQb